Determining Genre of Top 2023 Songs using Musical Characteristics

Our goal is to identify the genre of a given song based on its characteristics. Spotify is a popular streaming service for music with over 500 million users. The dataset we chose provides the most streamed spotify songs of 2023 with relevant information about each song. The data set is "Most Streamed Spotify Songs 2023" which has over 900 songs as observations. Each song has a track name, artist name, artist count, released year, released month, released day, number of playlists it appears in (Spotify, Apple Music and Deezer), ranking on the top charts (Spotify, Apple Music, Deezer, Shazam), number of streams, beats per minute, key of the song, mode, danceability, valence, energy, acousticness, instrumentalness, liveness and speechiness. For the sake of our question we will only be using six characteristics of the song to determine it’s genre: bpm, key, danceability, energy, acousticness, and speechiness. We have determined that these are the most pertinent vectors to examine in order to determine its genre based solely on the sound characteristics. Each genre has certain sounds which we will use to hopefully group it successfully to its genre. 

In [2]:
library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)
source('tests.R')
source("cleanup.R")


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks read

ERROR: Error in file(filename, "r", encoding = encoding): cannot open the connection


In [27]:
#url <- "https://raw.githubusercontent.com/nori-2004/data-science-group-project/main/spotify-2023.csv"
#spotify <- read_csv(url)
#spotify_selected <- select(spotify, track_name, bpm, key, streams, `danceability_%`, `energy_%`, `acousticness_%`, `speechiness_%`)
#spotify_selected

In [28]:
#spotify_split <- initial_split(spotify_selected, prop = 0.75)
#spotify_train <- training(spotify_split)   
#spotify_test <- testing(spotify_split)